In [1]:
!git clone https://github.com/andreazenotto/tempProjectRepo.git

Cloning into 'tempProjectRepo'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 102 (delta 46), reused 80 (delta 24), pack-reused 0 (from 0)
Receiving objects: 100% (102/102), 18.01 KiB | 9.00 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [2]:
%%capture
!apt update && apt install -y openslide-tools
!pip install openslide-python

In [3]:
import gdown
import os
from tqdm import tqdm
import pandas as pd
import sys

sys.path.append('tempProjectRepo/mesothelioma_project/src')
from wsi_utils import load_wsi, extract_patches, count_patches

In [4]:
data_csv = "tempProjectRepo/mesothelioma_project/data/raw-data.csv"
output_dir = "tempProjectRepo/mesothelioma_project/data/patches"

diagnosis_map = {"E": "epithelioid", "S": "sarcomatoid", "B": "biphasic"}

df = pd.read_csv(data_csv, delimiter=r"\s+")
start_idx = 0
end_idx = len(df)

## Finetuning satTresh on the patches of the first slide

In [ ]:
slide = load_wsi("M-1.ndpi")
values = [23, 25, 27, 29, 30]

for satThresh in values:
    print(f"Tresh = {satThresh} -> {count_patches(slide, "patches_test", level=1, thresh=satThresh)}")

## Download wsi dataset

In [5]:
import os
import shutil

!kaggle datasets download -d andreazenotto/wsi-part1
!unzip -q "wsi-part1.zip"
os.remove("wsi-part1.zip")
!mkdir -p wsi
shutil.move("wsi_part1", "wsi")
shutil.rmtree("wsi_part1")

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'
unzip:  cannot find or open wsi-part1.zip, wsi-part1.zip.zip or wsi-part1.zip.ZIP.


FileNotFoundError: [Errno 2] No such file or directory: 'wsi-part1.zip'

In [ ]:
!kaggle datasets download -d xxx/wsi-part2
!unzip -q "wsi-part2.zip"
os.remove("wsi-part2.zip")
!mkdir -p wsi
shutil.move("wsi_part2", "wsi")
shutil.rmtree("wsi_part2")

In [ ]:
!kaggle datasets download -d xxx/wsi-part3
!unzip -q "wsi-part3.zip"
os.remove("wsi-part3.zip")
!mkdir -p wsi
shutil.move("wsi_part3", "wsi")
shutil.rmtree("wsi_part3")

## Count patches

In [ ]:
def topk_wsi_names(dict_count_patches, k=5):

    # print(dict_count_patches)

    # Idea to realize the function
    # get all names, then create three masks:
    # - e_mask, b_mask, s_mask
    # for key in keys key.split('_')[1], then store info about index in the corresponding mask

    d_keys = list(dict_count_patches.keys())

    # Mask to each label
    e_mask = np.zeros(len(d_keys))
    b_mask = np.zeros(len(d_keys))
    s_mask = np.zeros(len(d_keys))
    for index in range(len(d_keys)):
        if 'epithelioid' in d_keys[index]:
            e_mask[index] = 1
        elif 'biphasic' in d_keys[index]:
            b_mask[index] = 1
        else:
            s_mask[index] = 1

    # flatter the dict to a list of values ( order is immutated )
    np_dict = np.array(list(dict_count_patches.values()))

    # get top k featuers for each label ( index i position is the i-th 1 element in mask )
    e_topk_indices = np.argpartition(np_dict[e_mask == 1], -k)[-k:]
    b_topk_indices = np.argpartition(np_dict[b_mask == 1], -k)[-k:]
    s_topk_indices = np.argpartition(np_dict[s_mask == 1], -k)[-k:]

    # compute original position with respect to the original dictionary
    combinations = [(e_topk_indices, e_mask, 'e'), (b_topk_indices, b_mask, 'b'), (s_topk_indices, s_mask, 's')]

    # devo prendere, tra gli indici marcati come 1 in e_mask, gli indici in posizione 1 e in posizione 2 ( scarto prendo prendo scarto)
    topk_indices = []
    for topk_list, mask, label in combinations:
      indices_of_interest = []
      #  print(topk_list, mask, label)
      for i in range(len(topk_list)):
          index = topk_list[i]
          for j in range(len(mask)):
              if mask[j] == 1:
                  if index == 0:
                    indices_of_interest.append(j)
                    break
                  else:
                    index -= 1
                    continue
              else:
                  continue

      # collecting all indices to easy extract only that ones in the next cell
      topk_indices.extend(indices_of_interest)
      print(f"Top {k} indices for label {label}:", end='\t')
      for index in indices_of_interest:
        print(d_keys[index], end= '\t')
      print()

      return topk_indices

In [ ]:
count_dict = {}
start_idx = 1
end_idx = 40

for _, row in tqdm(df.iloc[start_idx:end_idx].iterrows(), total=len(df)):
    filename = row['filename']
    gdrive_id = row['id']
    diagnosis_code = row['diagnosis']
    diagnosis_name = diagnosis_map[diagnosis_code]

    name = filename.split(".")[0] + " - " + diagnosis_name
    count_dict[name] = 0

    slide_id = os.path.splitext(filename)[0]
    slide_output_dir = os.path.join(output_dir, diagnosis_name, slide_id)

    # Load and process the WSI
    slide = load_wsi(os.path.join("wsi", filename))
    count_dict[filename] = count_patches(slide, slide_output_dir, level=1)

topk_indices = topk_wsi_names(count_dict, k=5)

## Segmentation and Patching

In [ ]:
for _, row in tqdm(df.iloc[topk_indices].iterrows(), total=len(df)):
    filename = row['filename']
    gdrive_id = row['id']
    diagnosis_code = row['diagnosis']
    diagnosis_name = diagnosis_map[diagnosis_code]

    slide_id = os.path.splitext(filename)[0]
    slide_output_dir = os.path.join(output_dir, diagnosis_name, slide_id)

    # Load and process the WSI
    slide = load_wsi(os.path.join("wsi", filename))
    extract_patches(slide, slide_output_dir, level=1, threshold=30)

In [ ]:
from google.colab import files
import shutil

# Percorso della cartella da scaricare
folder_path = "tempProjectRepo/mesothelioma_project/data/patches"

# Nome del file zip da creare
zip_filename = "patches.zip"

# Comprimi la cartella
shutil.make_archive(zip_filename.replace(".zip", ""), 'zip', folder_path)

# Scarica il file zip
files.download(zip_filename)